In [ ]:
print('Wellcome to Obesity_Classification_Model.')

Wellcome to Obesity_Classification_Model.


In [ ]:
print('Wellcome to Obesity_Classification_Model')

Wellcome to Obesity_Classification_Model


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [ ]:
# Calculate BMI
df['BMI'] = df['Weight'] / (df['Height'] ** 2)

# Add BMI categories based on WHO standards
bmi_bins = [0, 18.5, 25, 30, 35, 40, 100]
bmi_labels = ['Underweight', 'Normal', 'Overweight', 'Obese_I', 'Obese_II', 'Obese_III']
df['BMI_Category'] = pd.cut(df['BMI'], bins=bmi_bins, labels=bmi_labels)

# Create binary columns for each BMI category with 0/1 values
for category in bmi_labels:
    df[f'BMI_{category}'] = (df['BMI_Category'] == category).astype(int)

# Drop the original categorical column
df = df.drop('BMI_Category', axis=1)
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,...,no,Public_Transportation,Normal_Weight,24.386526,0,1,0,0,0,0
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,...,Sometimes,Public_Transportation,Normal_Weight,24.238227,0,1,0,0,0,0
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,...,Frequently,Public_Transportation,Normal_Weight,23.765432,0,1,0,0,0,0
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,...,Frequently,Walking,Overweight_Level_I,26.851852,0,0,1,0,0,0
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,...,Sometimes,Public_Transportation,Overweight_Level_II,28.342381,0,0,1,0,0,0


In [ ]:
print(df.info())  # Check data types
print(df.isnull().sum())  # Check missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [ ]:
# Define explicit numerical mappings for ordinal categorical features (CAEC, CALC)
# Maps text categories to integers while preserving their natural order:
# 'no' (never) → 0
# 'Sometimes' (occasionally) → 1
# 'Frequently' (regularly) → 2
# 'Always' (constantly) → 3
# This ordinal encoding maintains meaningful progression for machine learning models
ordinal_mappings = {
    'CAEC': {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3},
    'CALC': {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
}

# Apply the ordinal mappings to convert text categories to numerical values
for col, mapping in ordinal_mappings.items():
    df[col] = df[col].map(mapping)

# Show first rows to verify the ordinal encoding was applied correctly
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,1,no,...,0,Public_Transportation,Normal_Weight,24.386526,0,1,0,0,0,0
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,1,yes,...,1,Public_Transportation,Normal_Weight,24.238227,0,1,0,0,0,0
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,1,no,...,2,Public_Transportation,Normal_Weight,23.765432,0,1,0,0,0,0
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,1,no,...,2,Walking,Overweight_Level_I,26.851852,0,0,1,0,0,0
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,1,no,...,1,Public_Transportation,Overweight_Level_II,28.342381,0,0,1,0,0,0


In [ ]:
# Binary encoding for Gender column
from sklearn.preprocessing import LabelEncoder

for col in ['Gender', 'MTRANS']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,0,21.0,1.62,64.0,yes,no,2.0,3.0,1,no,...,0,3,Normal_Weight,24.386526,0,1,0,0,0,0
1,0,21.0,1.52,56.0,yes,no,3.0,3.0,1,yes,...,1,3,Normal_Weight,24.238227,0,1,0,0,0,0
2,1,23.0,1.80,77.0,yes,no,2.0,3.0,1,no,...,2,3,Normal_Weight,23.765432,0,1,0,0,0,0
3,1,27.0,1.80,87.0,no,no,3.0,3.0,1,no,...,2,4,Overweight_Level_I,26.851852,0,0,1,0,0,0
4,1,22.0,1.78,89.8,no,no,2.0,1.0,1,no,...,1,3,Overweight_Level_II,28.342381,0,0,1,0,0,0


In [ ]:
# Encoding - Binary Features (Yes/No → 0/1)
binary_features = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
binary_mapping = {'no': 0, 'yes': 1}

for feature in binary_features:
    df[feature] = df[feature].map(binary_mapping)
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,0,21.0,1.62,64.0,1,0,2.0,3.0,1,0,...,0,3,Normal_Weight,24.386526,0,1,0,0,0,0
1,0,21.0,1.52,56.0,1,0,3.0,3.0,1,1,...,1,3,Normal_Weight,24.238227,0,1,0,0,0,0
2,1,23.0,1.80,77.0,1,0,2.0,3.0,1,0,...,2,3,Normal_Weight,23.765432,0,1,0,0,0,0
3,1,27.0,1.80,87.0,0,0,3.0,3.0,1,0,...,2,4,Overweight_Level_I,26.851852,0,0,1,0,0,0
4,1,22.0,1.78,89.8,0,0,2.0,1.0,1,0,...,1,3,Overweight_Level_II,28.342381,0,0,1,0,0,0


In [ ]:
# Define the correct order of obesity categories from least to most severe
obesity_order = [
    'Insufficient_Weight',    # Underweight
    'Normal_Weight',          # Healthy weight
    'Overweight_Level_I',     # Mild overweight
    'Overweight_Level_II',    # Moderate overweight
    'Obesity_Type_I',         # Class I obesity
    'Obesity_Type_II',        # Class II obesity
    'Obesity_Type_III'        # Class III obesity (most severe)
]

# Convert the NObeyesdad column to an ordered categorical variable:
# 1. First convert to Pandas category type
# 2. Set the specific category order we defined above
# 3. Convert categories to numerical codes (0-6) while preserving order
df['NObeyesdad'] = (df['NObeyesdad']
                   .astype('category')
                   .cat.set_categories(obesity_order, ordered=True)
                   .cat.codes)

# Show the first few rows to verify the transformation
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,0,21.0,1.62,64.0,1,0,2.0,3.0,1,0,...,0,3,1,24.386526,0,1,0,0,0,0
1,0,21.0,1.52,56.0,1,0,3.0,3.0,1,1,...,1,3,1,24.238227,0,1,0,0,0,0
2,1,23.0,1.80,77.0,1,0,2.0,3.0,1,0,...,2,3,1,23.765432,0,1,0,0,0,0
3,1,27.0,1.80,87.0,0,0,3.0,3.0,1,0,...,2,4,2,26.851852,0,0,1,0,0,0
4,1,22.0,1.78,89.8,0,0,2.0,1.0,1,0,...,1,3,3,28.342381,0,0,1,0,0,0


In [ ]:
# Separate Features (X) and Target (y)
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

In [ ]:
# Define Numerical & Categorical Column
numerical_features = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE','BMI']
categorical_features = list(set(X.columns) - set(numerical_features))

In [ ]:
# Apply StandardScaler to Numerical Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', 'passthrough', categorical_features)  # Already encoded
    ])

X_processed = preprocessor.fit_transform(X)

df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,CALC,MTRANS,NObeyesdad,BMI,BMI_Underweight,BMI_Normal,BMI_Overweight,BMI_Obese_I,BMI_Obese_II,BMI_Obese_III
0,0,21.0,1.62,64.0,1,0,2.0,3.0,1,0,...,0,3,1,24.386526,0,1,0,0,0,0
1,0,21.0,1.52,56.0,1,0,3.0,3.0,1,1,...,1,3,1,24.238227,0,1,0,0,0,0
2,1,23.0,1.80,77.0,1,0,2.0,3.0,1,0,...,2,3,1,23.765432,0,1,0,0,0,0
3,1,27.0,1.80,87.0,0,0,3.0,3.0,1,0,...,2,4,2,26.851852,0,0,1,0,0,0
4,1,22.0,1.78,89.8,0,0,2.0,1.0,1,0,...,1,3,3,28.342381,0,0,1,0,0,0


In [ ]:
#  Split Data into Train & Test Sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)


In [ ]:
# =============================================
# 2. MODEL TRAINING AND EVALUATION
# =============================================

from sklearn.metrics import accuracy_score, classification_report

# Initialize dictionary to store accuracy results
results = {}

In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Remove the deprecated 'multi_class' parameter (let sklearn handle it automatically)
# 2. Increase max_iter and add regularization
lr = LogisticRegression(solver='lbfgs',
                       max_iter=5000,  # Increased from 1000
                       C=0.1,          # Added regularization
                       random_state=42)

# 3. Ensure your data is properly scaled (should already be done with StandardScaler)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
results['Logistic Regression'] = accuracy_score(y_test, y_pred)

In [ ]:
# -------------------------
# 2.2 Support Vector Machine
# -------------------------
from sklearn.svm import SVC
svm = SVC(kernel='rbf',   # Radial Basis Function kernel
          C=1.0,         # Regularization parameter
          gamma='scale')  # Auto-scale gamma
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
results['SVM'] = accuracy_score(y_test, y_pred)

In [ ]:
# -------------------------
# 2.3 k-Nearest Neighbors
# -------------------------
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)  # Default 5 neighbors
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
results['k-NN'] = accuracy_score(y_test, y_pred)

In [ ]:

# -------------------------
# 2.4 Naive Bayes
# -------------------------
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()  # Assumes Gaussian distribution of features
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
results['Naive Bayes'] = accuracy_score(y_test, y_pred)

In [ ]:
# -------------------------
# 2.5 Neural Network (MLP)
# -------------------------
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,),  # Single hidden layer with 100 neurons
                   max_iter=500)               # Maximum training iterations
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
results['Neural Network'] = accuracy_score(y_test, y_pred)

In [ ]:
# =============================================
# 3. RESULTS COMPARISON
# =============================================

print("\n=== MODEL ACCURACY COMPARISON ===")
print("(Sorted from highest to lowest)")
print("---------------------------------")
for model, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:20} {score:.4f}")  # Format: Model name + accuracy to 4 decimal places

# Identify and report the best model
best_model = max(results, key=results.get)
print("\n=== BEST MODEL DETAILS ===")
print(f"Model: {best_model}")
print(f"Accuracy: {results[best_model]:.4f}")






=== MODEL ACCURACY COMPARISON ===
(Sorted from highest to lowest)
---------------------------------
Neural Network       0.9622
Naive Bayes          0.9504
Logistic Regression  0.9480
k-NN                 0.9102
SVM                  0.5556

=== BEST MODEL DETAILS ===
Model: Neural Network
Accuracy: 0.9622
